In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import utils
import cv2 as cv
import os
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [7]:
# Path to test database
database_path = "path to database"
database = pd.read_csv(database_path)

In [ ]:
# this code extracts the contents of multiple zip files located in a specified directory
# (dir_path) and extracts them into a different directory (new_dir). It iterates over unique
# names in a database, assumes the zip files have the same name as the "name" column, and 
# extracts the contents of each zip file into the target directory. {new_dir}
#└── {WSI_Name}
#    ├── {WSI_Name}_img
#    │   └── img_patches
#    └── {WSI_Name}_mask
#        └── mask_patches
#------------------------------------------------------#

import zipfile
names = database.name.unique()
dir_path = "path of the directory the ziped data"

new_dir = "path of the directory where the zipped data will be extracted "
for name in names:
    zip_file_path = dir_path+ name + '.zip'  # Assuming the zip files have the same name as the "name" column

    if os.path.exists(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(new_dir)
            print(f"Successfully extracted {zip_file_path}")
    else:
        print(f"Zip file {zip_file_path} does not exist")

In [ ]:
# This function is used to calculate mean IoU scores inside the model.
# It is important to define this function in order to be able to load the model
#------------------------------------------------------#
def mean_iou_manual_tf(y_true, y_pred):
    y_pred = tf.round(y_pred)
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    ious = []
    for val in [0, 1]:  # Assuming binary segmentation
        y_true_binary = tf.cast(tf.equal(y_true, val), tf.float32)
        y_pred_binary = tf.cast(tf.equal(y_pred, val), tf.float32)

        intersection = tf.reduce_sum(y_true_binary * y_pred_binary)
        union = tf.reduce_sum(y_true_binary) + tf.reduce_sum(y_pred_binary) - intersection
        iou = intersection / union
        ious.append(iou)

    return tf.reduce_mean(ious)

utils.get_custom_objects()['mean_iou_manual_tf'] = mean_iou_manual_tf

In [ ]:
unet_model = tf.keras.models.load_model("path to the model")


In [ ]:
# This function loop through all the images and generate batch of images for testing
#------------------------------------------------------#
dir_path = "path to the extracted patches"

def test_generator(data, batch_size):

        print("entered While")

        for start in range(0, len(data), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(data))
            df_batch = data[start:end]
            for _, row in df_batch.iterrows():
                img = cv.imread(dir_path + row['img_path'])
                mask = cv.imread(dir_path + row['mask_path'], cv.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Image not loaded: '{row['image_name']}'")
                    continue

                if mask is None:
                    print(f"Mask not loaded: {row['mask_name']}")
                    continue

                img = img / 255.  # Normalize image

                x_batch.append(img)
                y_batch.append(mask)
            yield np.array(x_batch), np.array(y_batch)

In [ ]:
cancered = database[database.label == 1]
partially_cancered = database[database.label == -1]
benign = database[database.label == 0]

In [ ]:
# The model is tested through all test data and the final results are printed
batch_size = 16
threshold = 0.5
test_gen = test_generator(cancered, batch_size)
# Iterate over the evaluation batches
overall_mean_iou = 0
overall_mean_f1 = 0
overall_mean_recall = 0
overall_mean_precision = 0
overall_mean_accuracy = 0
total_batches = 0

for images, masks in test_gen:
    # Perform inference with your model
    predictions = unet_model.predict(images)
    predictions = (predictions > threshold).astype(np.float32)

    masks = masks.flatten()
    predictions = predictions.flatten()

    # Calculate metrics for the batch
    batch_iou = mean_iou_manual_tf(masks, predictions)
    batch_f1 = f1_score(masks, predictions, zero_division = 1)
    batch_recall = recall_score(masks, predictions, zero_division = 1)
    batch_precision = precision_score(masks, predictions, zero_division = 1)
    batch_accuracy = accuracy_score(masks, predictions)

    # Accumulate batch results
    overall_mean_iou += batch_iou
    overall_mean_f1 += batch_f1
    overall_mean_recall += batch_recall
    overall_mean_precision += batch_precision
    overall_mean_accuracy += batch_accuracy
    total_batches += 1

    # Print batch results
    print("Batch", total_batches)
    print("Mean IoU:", batch_iou)
    print("Accuracy:", batch_accuracy)
    print("F1 Score:", batch_f1)
    print("Recall:", batch_recall)
    print("Precision:", batch_precision)

# Calculate overall means
overall_mean_iou /= total_batches
overall_mean_f1 /= total_batches
overall_mean_recall /= total_batches
overall_mean_precision /= total_batches
overall_mean_accuracy /= total_batches

# Print the results
print("Cancered")
print("Overall Mean IoU:", overall_mean_iou)
print("Overall Mean F1 Score:", overall_mean_f1)
print("Overall Mean Recall:", overall_mean_recall)
print("Overall Mean Precision:", overall_mean_precision)
print("Overall Mean Accuracy:", overall_mean_accuracy)


In [ ]:
# Shows examples of the prediction
#----------------------------------------------------------------------#
import matplotlib.pyplot as plt
import numpy as np

batch_size = 32
test_gen = test_generator(val_data, batch_size)

# Get a batch of data
images, masks = next(test_gen)
threshold = 0.5
# Get predictions for this batch
predictions = unet_model_3.predict(images)
predictions = (predictions > threshold).astype(np.float32)
fig, axes = plt.subplots(batch_size, 3, figsize=(20, 10 * batch_size))

for idx in range(batch_size):
    # Show original image
    axes[idx, 0].imshow(images[idx])
    axes[idx, 0].set_title('Original Image')

    # Show ground truth mask
    axes[idx, 1].imshow(masks[idx], cmap='gray')
    axes[idx, 1].set_title('Ground Truth Mask')

    # Show predicted mask

    axes[idx, 2].imshow(predictions[idx], cmap='gray')
    axes[idx, 2].set_title('Predicted Mask')

plt.tight_layout()
plt.show()

